# DataFrameExtractor


<div class="alert alert-warning">

**Warning**: This section is a bit technical and many users won't need this functionality. Also, it is a bit experimental and the API may change in future versions. Proceed with caution.

</div>

The callables [`picks_to_df`, `events_to_df`](../datastructures/events_to_pandas.ipynb), and [`inventory_to_df`](../datastructures/stations_to_pandas.ipynb) are instances of `DataFrameExtractor`, which provides an extensible and customizable way for creating callables that extract `DataFrames` from arbitrary objects.

To demonstrate, let's create a new extractor to put arrival objects in the Crandall catalog into a dataframe. We can then join this table together with the picks table to do some (possibly) interesting things.

In [ ]:
import obspy

import obsplus

crandall = obsplus.load_dataset('crandall_test')
cat =crandall.event_client.get_events()
print(cat)

We will start by initializing the extractor with a list of expected columns and data types. This is optional, but helps ensure the output dataframe has a consistent shape and data type. The [arrival documentation](https://docs.obspy.org/packages/autogen/obspy.core.event.origin.Arrival.html) may be useful to understand these. Rather than collecting all the data contained in the `Arrival` instances, we will only create a few columns of interest.

In [ ]:
from collections import OrderedDict

import obspy.core.event as ev

# declare datatypes (order to double as required columns)
dtypes = OrderedDict(
    resource_id=str, 
    pick_id=str, 
    event_id=str,
    origin_id= str, 
    phase=str, 
    time_correction=float, 
    distance=float, 
    time_residual=float,                
    time_weight=float,
)

# init the DataFrameExtractor
arrivals_to_df = obsplus.DataFrameExtractor(ev.Arrival, required_columns=list(dtypes), 
                                            dtypes=dtypes)

Next we define some "extractors". These are callables that will take an `Arrival` instance and return the desired data. The extractors can return:

1. A `dict` of values where each key corresponds to a column name and each value is the row value of that column for the current object.

2. Anything else, which is interpreted as the row value, and the column name is obtained from the function name.

In [ ]:
# an extractor which returns a dictionary
@arrivals_to_df.extractor
def _get_basic(arrival):
    out = dict(
        resource_id=str(arrival.resource_id),
        pick_id=str(arrival.pick_id),
        time_correction=arrival.time_correction,
        distance=arrival.distance,
        time_residual=arrival.time_residual,
        time_weight=arrival.time_weight,
    )
    return out


# an extractor which returns a single value
@arrivals_to_df.extractor
def _get_phase(arrival):
    return arrival.phase

Notice, however, that we have no way of extracting information from the parent `Origin` or `Event` objects. Also, the extractor doesn't yet know how to find the arrivals in a `Catalog` object. To accomplish both these tasks, we can define the types of data the extractor can operate on, and inject the event and origin data into arrival rows:

In [ ]:
@arrivals_to_df.register(obspy.Catalog)
def _get_arrivals_from_catalogs(cat):
    arrivals = []  # a list of arrivals
    extras = {}  # dict of data to inject to arrival level
    for event in cat:
        for origin in event.origins:
            arrivals.extend(origin.arrivals)
            data = dict(event_id=event.resource_id, origin_id=origin.resource_id)
            # use arrival id to inject extra to each arrival row
            extras.update({id(x): data for x in origin.arrivals})
    return arrivals_to_df(arrivals, extras=extras)

Great, now let's try out the extractor!

In [ ]:
df = arrivals_to_df(cat)
df.head()

In [ ]:
df.phase.value_counts()

Looks good, however, what if we only wanted P phases? The easiest thing to do is simply filter the dataframe, but, for demonstration, let's modify our phase extractor so that any row that is not a P phase is skipped. This is done by raising a `SkipRow` exception which is an attribute of the `DataFrameExtractor`.

In [ ]:
@arrivals_to_df.extractor
def _get_phase(arrival):
    phase = arrival.phase
    if phase.upper() != 'P':
        raise arrivals_to_df.SkipRow
    return phase

In [ ]:
df = arrivals_to_df(cat)
print(df.phase.value_counts())

Now we can get a picks dataframe and perform a left join on the phases:

In [ ]:
# get picks and filter out non-P phases
picks = obsplus.picks_to_df(cat)
picks = picks[picks.phase_hint.str.upper() == "P"]

In [ ]:
df_merged = df.merge(picks, how='left', right_on='resource_id', left_on='pick_id')

In [ ]:
df_merged.head()

In [ ]:
df_merged.columns

We can also calculate how often the `phase` attribute in the arrival is different from the `phase_hint` in the pick, which could indicate a quality issue.

In [ ]:
# calculate fraction of phase_hints that match phase
(df_merged['phase'] == df_merged['phase_hint']).sum() / len(df_merged)